# Named Entity Recognition for dataset names

## Date: March 9, 2020

In [1]:
!prodigy dataset dataset_name 'Seed terms for DATASET label'


✘ 'dataset_name' already exists in database SQLite



In [ ]:
!prodigy terms.teach dataset_name en_core_web_lg --seeds ''

In [43]:
import os
from os import listdir
import json
import pandas as pd
from os.path import isfile, join
import nltk

Getting the list of file names:

In [3]:
path = 'labeled_sample'
nme = [f for f in listdir(path) if isfile(join(path, f))]

Getting the data from the json files

In [4]:
txt = []
lbl = []
for nm in nme:
    try:
        with open('labeled_sample/{}'.format(nm), encoding='windows-1252') as f:
            dct = json.load(f)
            txt.append(list(dct.keys())[0])
            lbl.append(list(dct.values())[0])
    except: pass

In [5]:
d = {
    'text': txt,
    'label': lbl
}

In [6]:
dtf = pd.DataFrame(d, columns=['text', 'label'])

In [7]:
dtf.head(3)

,text,label
0,International Journal of Commun...,1
1,International Journal of Computer Applications...,0
2,816. Conclusions This thesis described the t...,0


Getting only the positive labels:

In [15]:
dtf_true = dtf[dtf.label==1].reset_index(drop=True)

In [19]:
dtf_true.loc[6, 'text']

'PAGE 38MULTIFAMILY GREEN BUILDING GUIDELINESMEASURE AA7PASSIVE SOLAR DESIGN, DAYLIGHTING AND NATURAL VENTILATIONConsiderations for ResidentsResidents may beneﬁt from reduced heating and cooling bills and better indoor environmental quality. Teachresidents about the strategies used so that they don’t unintentionally circumvent the design. A throw rug, for example, will reduce a mass ﬂoor’s ability to store heat. Cost and Cost EffectivenessBENEFITCOSTThe strategies in this measure may increase design time. Passive solar design may increase material costs for items such as shading devices or extra concrete or drywall for thermal mass. However, passive design strategies are probably the best way to reduce ﬁrst costs associated with system sizing (Systems: H0–Heating Equipment) and provide ongoing savings throughout the building’s life. A basic level of daylighting and natural ventilation can usually be provided for no increase over standard construction costs. Skylights, clerestories, tal

In [9]:
dtf_true.shape

(50, 2)

Creating a set a subsets where we we have the sentence with the mention of "%data%" plus the sentence before (if it exists), plus the sentence after (if it exists).

- Extracting one sentence before and one sentence after the mention of data (3 sentences subset)

In [29]:
txt = dtf_true.loc[6, 'text']
a_list = nltk.tokenize.sent_tokenize(txt)

In [33]:
sub = []
def get_three_sentences_subsets(txt):
    stc = nltk.tokenize.sent_tokenize(txt)
    for i, st in enumerate(stc):
        if 'data' in st and i==0:
            sub.append(' '.join([st, stc[1]]))
        elif 'data' in st and i!=len(stc)-1:
            sub.append(' '.join([stc[i-1], st, stc[i+1]]))
        elif 'data' in st and i==len(stc)-1:
            sub.append(' '.join([stc[i-1], st]))
        else: pass
    return

In [35]:
list(map(get_three_sentences_subsets, list(dtf_true.text)))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

We end up with 148 subsets:

In [37]:
len(sub)

148

Example of a subset that we have:

In [40]:
sub[0]

'Then it uses the pre-processed signal to extract features from the ECG signal automatically. The ECG signals for the present study are collected from MIT/BIH database via Physionet database and stored in a text format. The MIT-BIH database contains different types of Electrocardiogram signals including both abnormal Electrocardiograms and normal Electrocardiograms, which are sampled at different rates.'

Now, we want to identify for each of the subsets (if it exists):
- the name of the mentioned dataset
- the source of the mentioned dataset

We save our subset into a json file:

In [41]:
json_list = [{'text': txt} for txt in sub]

In [44]:
if not os.path.exists('prodigy'):
    os.mkdir('prodigy')

In [46]:
with open('prodigy/ner_text_to_label.json', 'w') as f:
    json.dump(json_list, f)

Loading the data:

In [48]:
#with open('prodigy/ner_text_to_label.json', 'w') as f:
#    dat_to_label = json.load(f)

In [ ]:
!prodigy ner.manual json_list en_core_web_sm /tmp/ner_datat.dump --loader txt --label dataset_name

Using 1 label(s): dataset_name
Added dataset json_list to database SQLite.

✨  Starting the web server at http://localhost:8080 ...
Open the app in your browser and start annotating!


✘ Can't find file /tmp/ner_datat.dump
/private/tmp/ner_datat.dump

